In [ ]:
# %load_ext autoreload
# %autoreload 2
# import sys; sys.path.append('/future/u/okhattab/repos/public/stanfordnlp/dspy')

import dspy
from dspy.evaluate import Evaluate
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
from dspy.modeling import JSONBackend

In [ ]:
gms8k = GSM8K()

backend = JSONBackend(model="gpt-3.5-turbo", params={"max_tokens": 500, "temperature": 0.5, "num_retries": 5})
dspy.settings.configure(backend=backend)

trainset, devset = gms8k.train, gms8k.dev

In [ ]:
NUM_THREADS = 4
evaluate = Evaluate(devset=devset[:], metric=gsm8k_metric, num_threads=NUM_THREADS, display_progress=True, display_table=0)

In [ ]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought("question -> answer")
    
    def forward(self, question):
        return self.prog(question=question)

In [ ]:
RUN_FROM_SCRATCH = True

if RUN_FROM_SCRATCH:
    config = dict(max_bootstrapped_demos=5, max_labeled_demos=8, num_candidate_programs=5, num_threads=NUM_THREADS)
    teleprompter = BootstrapFewShotWithRandomSearch(metric=gsm8k_metric, **config)
    cot_bs = teleprompter.compile(CoT(), trainset=trainset, valset=devset)
    # cot_bs.save('turbo_8_8_10_gsm8k_200_300.json')
else:
    cot_bs = CoT()
    cot_bs.load('turbo_8_8_10_gsm8k_200_300.json')

In [ ]:
evaluate(cot_bs, devset=devset[:])

In [ ]:
print(backend.history[-1].prompt.to_str())